In [ ]:
import pandas as pd
from pruning_lfs import prune_lfs
import re
import pickle

In [ ]:
manualLF2C = pd.read_excel("/workspace/datasets/manualLF2C.xlsx")
remaining_LFs = prune_lfs(CHOSEN_DS='eval')

animals = ["dog", "fish", "sheep", "bunny", "octopus", "roadrunner", "okapi", "anisakis", "bat", "leopard" ]

In [ ]:
thresless_LFs = {}

for index, row in manualLF2C.iterrows():
    curr_meth = row['Methode']
    pts_left_brackets = curr_meth.split('{')

    cryptic_name = pts_left_brackets[0]
    for pt in pts_left_brackets:
        pts_right_brackets = pt.split('}')

        if len(pts_right_brackets) > 1:
           cryptic_name = cryptic_name + 'XXX' + pts_right_brackets[1]
    
    thresless_LFs[cryptic_name] = curr_meth


In [ ]:
remaining_to_thresless = {}
clustered_to_cluster = {}

for lf in remaining_LFs:
    curr_lf = lf.name.replace('label=1', 'label=NOT_SIMPLE').replace('label=0', 'label=SIMPLE')

    if 'label' not in curr_lf and 'SIMPLE' not in curr_lf:
        if curr_lf[-2:] == '_1':
            curr_lf = curr_lf[:-2] + '_NOT_SIMPLE'
        if curr_lf[-2:] == '_0':
            curr_lf = curr_lf[:-2] + '_SIMPLE'
        
    wo_percentage = re.sub('\d+\.\d+', 'XXX', curr_lf)
    curr_lf = re.sub('\d+', 'XXX', wo_percentage)
    
    for to_exclude in animals:
        curr_lf = curr_lf.replace('_' + to_exclude + '_', '_XXX_')

    if 'perc_more_than_XXX_characters' in curr_lf:
        curr_lf = curr_lf.replace('perc_more_than_XXX_characters', 'perc_more_than_8_characters')
    
    if curr_lf not in thresless_LFs.keys():
        print(curr_lf)
    
    remaining_to_thresless[lf.name] = manualLF2C[manualLF2C['Methode'] == thresless_LFs[curr_lf]]['cluster'].values[0]
    #remaining_to_thresless[lf.name] = thresless_LFs[curr_lf]

In [ ]:
overview = {} # how often do we find a LF belonging to Structural, Lexical, Pragmatic, Syntactic
labels_to_clusters = {}
final_clustering_of_manual_LFs = {} # use this to compare to automatically clustered LFs

for lf in remaining_to_thresless.keys():
    if remaining_to_thresless[lf] in overview:
        overview[remaining_to_thresless[lf]] = overview[remaining_to_thresless[lf]] + 1
    else:
        labels_to_clusters[remaining_to_thresless[lf]] = len(labels_to_clusters)
        overview[remaining_to_thresless[lf]] = 1

    final_clustering_of_manual_LFs[lf] = labels_to_clusters[remaining_to_thresless[lf]]

In [ ]:
with open('/workspace/datasets/clustering_manual_LFs.pkl', 'wb') as f:
        pickle.dump(final_clustering_of_manual_LFs, f)

In [ ]:
with open('/workspace/datasets/clustering_manual_LFs_with_CLUSTERS.pkl', 'wb') as f:
        pickle.dump(final_clustering_of_manual_LFs, f)